In [1]:
import pygame
import numpy as np

# Constants
WIDTH, HEIGHT = 1280, 720
CLOTH_WIDTH, CLOTH_HEIGHT = 20, 15  # Reduced for better performance
SPACING = 20
GRAVITY = np.array([0, 0.5])
FRICTION = 0.99
CONSTRAINT_ITERATIONS = 3
BALL_RADIUS = 20
BALL_SPEED = 10
BALL_FRICTION = 0.99
IMPACT_FORCE_MULTIPLIER = 20.0

pygame 2.2.0 (SDL 2.30.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Cloth Simulation")
clock = pygame.time.Clock()

class Point:
    def __init__(self, x, y, pinned=False):
        self.position = np.array([x, y], dtype=float)
        self.old_position = self.position.copy()
        self.acceleration = np.zeros(2)
        self.pinned = pinned

    def apply_force(self, force):
        self.acceleration += force

    def update(self):
        if not self.pinned:
            velocity = (self.position - self.old_position) * FRICTION
            self.old_position = self.position.copy()
            self.position += velocity + self.acceleration
            self.acceleration = np.zeros(2)

    def constrain(self, width, height):
        self.position = np.clip(self.position, [0, 0], [width, height])

class Stick:
    def __init__(self, point_a, point_b):
        self.point_a = point_a
        self.point_b = point_b
        self.length = np.linalg.norm(point_a.position - point_b.position)

    def update(self):
        diff = self.point_a.position - self.point_b.position
        distance = np.linalg.norm(diff)
        difference = (self.length - distance) / distance
        offset = diff * 0.5 * difference

        if not self.point_a.pinned:
            self.point_a.position += offset
        if not self.point_b.pinned:
            self.point_b.position -= offset

class Ball:
    def __init__(self, position, velocity, radius):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.radius = radius

    def update(self):
        self.velocity *= BALL_FRICTION
        self.position += self.velocity

    def draw(self, screen):
        pygame.draw.circle(screen, (255, 0, 0), self.position.astype(int), self.radius)

    def check_collision(self, point):
        dist = np.linalg.norm(self.position - point.position)
        if dist < self.radius:
            direction = (point.position - self.position) / dist
            displacement = self.radius - dist
            point.position += direction * displacement * 0.5
            self.position -= direction * displacement * 0.5
            impact_force = direction * np.linalg.norm(self.velocity) * IMPACT_FORCE_MULTIPLIER
            point.apply_force(impact_force)
            self.velocity = -self.velocity * 0.7  # Bounce effect

points = []
sticks = []

# Create cloth points
for y in range(CLOTH_HEIGHT):
    for x in range(CLOTH_WIDTH):
        pinned = (y == 0 and (x % (CLOTH_WIDTH // 10) == 0))
        points.append(Point(x * SPACING + (WIDTH - CLOTH_WIDTH * SPACING) // 2, y * SPACING + 50, pinned))

# Create cloth sticks
for y in range(CLOTH_HEIGHT):
    for x in range(CLOTH_WIDTH):
        if x < CLOTH_WIDTH - 1:
            sticks.append(Stick(points[y * CLOTH_WIDTH + x], points[y * CLOTH_WIDTH + x + 1]))
        if y < CLOTH_HEIGHT - 1:
            sticks.append(Stick(points[y * CLOTH_WIDTH + x], points[(y + 1) * CLOTH_WIDTH + x]))

# Variables for mouse interaction
selected_point = None
mouse_down = False
ball = None

# Simulation loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Left mouse button
                mouse_pos = np.array(event.pos, dtype=float)
                # Find the closest point to the mouse click
                for point in points:
                    if np.linalg.norm(point.position - mouse_pos) < SPACING:
                        selected_point = point
                        mouse_down = True
                        break
            elif event.button == 3:  # Right mouse button
                # Shoot a ball from the mouse position
                ball_start_pos = np.array(event.pos, dtype=float)
                ball_velocity = - (ball_start_pos - np.array([WIDTH // 2, HEIGHT // 2])) / np.linalg.norm(ball_start_pos - np.array([WIDTH // 2, HEIGHT // 2])) * BALL_SPEED
                ball = Ball(ball_start_pos, ball_velocity, BALL_RADIUS)
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:  # Left mouse button
                selected_point = None
                mouse_down = False

    if mouse_down and selected_point:
        mouse_pos = np.array(pygame.mouse.get_pos(), dtype=float)
        selected_point.old_position = mouse_pos - (selected_point.position - selected_point.old_position)
        selected_point.position = mouse_pos

    screen.fill((0, 0, 0))

    # Apply gravity to each point
    for point in points:
        point.apply_force(GRAVITY)

    # Update each point
    for point in points:
        point.update()

    # Apply constraints multiple times for stability
    for _ in range(CONSTRAINT_ITERATIONS):
        for stick in sticks:
            stick.update()
        for point in points:
            point.constrain(WIDTH, HEIGHT)

    # Draw sticks
    for stick in sticks:
        pygame.draw.line(screen, (255, 255, 255), stick.point_a.position, stick.point_b.position)

    # Update and draw ball
    if ball:
        ball.update()
        ball.draw(screen)
        for point in points:
            ball.check_collision(point)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
